In [1]:
import sympy as sp
from numpy import array
import numpy as np
from sympy import init_printing
init_printing(use_latex='mathjax')
import scipy


sqrt2 = np.sqrt(2)
sqrt3 = np.sqrt(3)
upper_triangle_idx  =np.triu_indices(5)

In [2]:

def remove_hydrostatic_dependence(data, transpose_data):
    # Get the rotation matrix with full precision


    d = 1./np.sqrt(6) * np.array( [[ sqrt2, sqrt2, sqrt2, 0, 0, 0  ],
                                   [ -1, -1, 2, 0, 0, 0],
                                   [-sqrt3, sqrt3, 0, 0, 0, 0],
                                   [0, 0, 0, 2*sqrt3, 0, 0],
                                   [0, 0, 0, 0, 2*sqrt3, 0],
                                   [0, 0, 0, 0, 0, 2*sqrt3] ])

    # Isolate stress
    stress_vectors = data[:,0:6]
    stress_vectors_T = transpose_data[:, 0:6]

    # # Shift the X_0 up a tiny bit
    # data[:,6] += 1e-6
    # transpose_data[:,6] += 1e-6

    # Rotate to natural basis
    stress_vectors = (d @ stress_vectors.T).T
    stress_vectors_T = (d @ stress_vectors_T.T).T
    #print(np.shape(stress_vectors_T), np.shape(transpose_data[:,3].T))

    # We assert that there should be hydrostatic stress independence. Therefore, we only solve for the 5x5 submatrix
    data = np.column_stack([stress_vectors[:,1:6], data[:,6].T ])
    transpose_data = np.column_stack( [stress_vectors_T[:, 1:6], transpose_data[:,6].T ])

    return data, transpose_data

In [3]:
# load data
hill_data_path = "../data_6x6/processed_data/hill48_bingo.csv"
hill_transposed_data_path = "../data_6x6/processed_data/hill48_bingo_transpose.csv"
data_pre = np.loadtxt(hill_data_path)
transposed_data_pre = np.loadtxt(hill_transposed_data_path)

In [4]:
data, transposed_data = remove_hydrostatic_dependence(data_pre, transposed_data_pre)
eqps_0_data = np.isclose(data[:,5],0.0)
#print(eqps_0_data)
data = data[eqps_0_data,:]
eqps_0_trans_data = np.isclose(transposed_data[:,5],0.0)
transposed_data = transposed_data[eqps_0_trans_data, :]
data, transposed_data

(array([[ 0.75951915, -2.4402084 ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-1.91389396,  2.19027893,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-3.1360807 , -0.56234133,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 3.35252597, -0.56234133,  0.        ,  0.        ,  0.        ,
          0.        ],
        [-0.32466792, -0.56234133, -3.64579972,  0.        ,  0.        ,
          0.        ],
        [-0.32466792, -0.56234133,  3.64579972,  0.        ,  0.        ,
          0.        ],
        [-0.32466792, -0.56234133,  0.        , -6.82066673,  0.        ,
          0.        ],
        [-0.32466792, -0.56234133,  0.        ,  6.82066673,  0.        ,
          0.        ],
        [-0.32466792, -0.56234133,  0.        ,  0.        , -9.64587939,
          0.        ],
        [-0.32466792, -0.56234133,  0.        ,  0.        ,  9.64587939,
          0.        ],
        [ 0.68181796,  2.30562

In [5]:
def objective(params):
    #print(params)
    # fill a P matrix that has the params:
    candidate_P = np.array( [[ params[0], params[1], 0, 0, 0 ],
                             [ params[1], params[2], 0, 0, 0],
                             [ 0, 0, params[3], 0, 0],
                             [ 0, 0, 0, params[4], 0],
                             [0, 0, 0, 0, params[5] ]
                             ] )
    #print(candidate_P)
    residual = np.zeros(len(data))
    for i, stress_vec in enumerate(data):
        sv = stress_vec[0:5]
        current_ys =  sv @ candidate_P @ sv 
        error = current_ys - 100

        residual[i] = error
        
    #print(scipy.linalg.norm(residual))
    return np.sum(np.abs(residual))
    


In [6]:
initial_guess = np.array([ 9/2, 0.0, 31/2, 2, 2, 1.0 ])
result = scipy.optimize.minimize(objective, initial_guess, method = 'SLSQP', options={'fatol':1e-6, 'maxiter':1e5})
print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.00015848498271964218
       x: [ 9.000e+00  1.732e+00  1.700e+01  7.000e+00  2.000e+00
            1.000e+00]
     nit: 37
     jac: [ 8.460e+01  3.734e+01 -8.046e+01  1.345e+02 -4.709e+02
            9.418e+02]
    nfev: 347
    njev: 37


C:\Users\brian\AppData\Local\Temp\ipykernel_18572\1889372463.py:2: OptimizeWarning: Unknown solver options: fatol
  result = scipy.optimize.minimize(objective, initial_guess, method = 'SLSQP', options={'fatol':1e-6, 'maxiter':1e5})


In [7]:
params = result.x
candidate_P = np.array( [[ params[0], params[1], 0, 0, 0 ],
                             [ params[1], params[2], 0, 0, 0],
                             [ 0, 0, params[3], 0, 0],
                             [ 0, 0, 0, params[4], 0],
                             [0, 0, 0, 0, params[5] ]
                             ] )
#print(candidate_P)
residual = np.zeros(len(data))
for i, stress_vec in enumerate(data):
    sv = stress_vec[0:5]
    current_ys =  sv @ candidate_P @ sv 
    error = current_ys - 100

    residual[i] = error
print(candidate_P)
print(residual)

[[ 9.00000049  1.73205109  0.          0.          0.        ]
 [ 1.73205109 16.99999984  0.          0.          0.        ]
 [ 0.          0.          6.99999992  0.          0.        ]
 [ 0.          0.          0.          1.99999997  0.        ]
 [ 0.          0.          0.          0.          1.00000005]]
[-1.68497127e-06 -1.30907357e-06  5.74611879e-06  4.38492768e-06
 -1.00286049e-06 -1.00286049e-06 -1.34581219e-06 -1.34581219e-06
  5.10991829e-06  5.10991829e-06  2.81810458e-07  2.78337058e-06
  4.38492768e-06  5.74611882e-06 -1.22281300e-06 -1.22281300e-06
 -1.57042665e-06 -1.57042665e-06  4.97306918e-06  4.97306918e-06
  2.78337055e-06  2.81810458e-07 -1.30907360e-06 -1.68497122e-06
 -9.39208306e-07 -9.39208306e-07 -1.29381566e-06 -1.29381566e-06
  5.38132802e-06  5.38132802e-06  1.26671897e-06  1.26671897e-06
 -1.70927594e-06 -1.70927594e-06  4.84318650e-06  4.84318650e-06
 -1.52636727e-06 -1.52636727e-06  4.79771572e-06  4.79771572e-06
  1.42789327e-06  1.42789327e-06 -